In [ ]:
!pip install transformers==4.41.0, datasets==3.1.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
  Attempting uninstall: tr

# Import

In [ ]:
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torch.nn.utils.rnn import pad_sequence
from tqdm import tqdm
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
import os
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
torch.manual_seed(42)
np.random.seed(42)

In [ ]:
class BERT():
  def __init__(self):
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    self.model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=4)
    self.model.to(self.device)

  def get_model(self):
    return self.model

  def get_tokenizer(self):
    return self.tokenizer

In [ ]:
from datasets import load_dataset
ds = load_dataset("community-datasets/per_sent")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/14.0k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/8.84M [00:00<?, ?B/s]

test_random-00000-of-00001.parquet:   0%|          | 0.00/1.60M [00:00<?, ?B/s]

test_fixed-00000-of-00001.parquet:   0%|          | 0.00/2.27M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.41M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/3355 [00:00<?, ? examples/s]

Generating test_random split:   0%|          | 0/579 [00:00<?, ? examples/s]

Generating test_fixed split:   0%|          | 0/827 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/578 [00:00<?, ? examples/s]

In [ ]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['DOCUMENT_INDEX', 'TITLE', 'TARGET_ENTITY', 'DOCUMENT', 'MASKED_DOCUMENT', 'TRUE_SENTIMENT', 'Paragraph0', 'Paragraph1', 'Paragraph2', 'Paragraph3', 'Paragraph4', 'Paragraph5', 'Paragraph6', 'Paragraph7', 'Paragraph8', 'Paragraph9', 'Paragraph10', 'Paragraph11', 'Paragraph12', 'Paragraph13', 'Paragraph14', 'Paragraph15'],
        num_rows: 3355
    })
    test_random: Dataset({
        features: ['DOCUMENT_INDEX', 'TITLE', 'TARGET_ENTITY', 'DOCUMENT', 'MASKED_DOCUMENT', 'TRUE_SENTIMENT', 'Paragraph0', 'Paragraph1', 'Paragraph2', 'Paragraph3', 'Paragraph4', 'Paragraph5', 'Paragraph6', 'Paragraph7', 'Paragraph8', 'Paragraph9', 'Paragraph10', 'Paragraph11', 'Paragraph12', 'Paragraph13', 'Paragraph14', 'Paragraph15'],
        num_rows: 579
    })
    test_fixed: Dataset({
        features: ['DOCUMENT_INDEX', 'TITLE', 'TARGET_ENTITY', 'DOCUMENT', 'MASKED_DOCUMENT', 'TRUE_SENTIMENT', 'Paragraph0', 'Paragraph1', 'Paragraph2', 'Paragraph3',

In [ ]:
print(ds["train"].features)
print(ds["train"][0])

{'DOCUMENT_INDEX': Value(dtype='int64', id=None), 'TITLE': Value(dtype='string', id=None), 'TARGET_ENTITY': Value(dtype='string', id=None), 'DOCUMENT': Value(dtype='string', id=None), 'MASKED_DOCUMENT': Value(dtype='string', id=None), 'TRUE_SENTIMENT': ClassLabel(names=['Negative', 'Neutral', 'Positive'], id=None), 'Paragraph0': ClassLabel(names=['Negative', 'Neutral', 'Positive'], id=None), 'Paragraph1': ClassLabel(names=['Negative', 'Neutral', 'Positive'], id=None), 'Paragraph2': ClassLabel(names=['Negative', 'Neutral', 'Positive'], id=None), 'Paragraph3': ClassLabel(names=['Negative', 'Neutral', 'Positive'], id=None), 'Paragraph4': ClassLabel(names=['Negative', 'Neutral', 'Positive'], id=None), 'Paragraph5': ClassLabel(names=['Negative', 'Neutral', 'Positive'], id=None), 'Paragraph6': ClassLabel(names=['Negative', 'Neutral', 'Positive'], id=None), 'Paragraph7': ClassLabel(names=['Negative', 'Neutral', 'Positive'], id=None), 'Paragraph8': ClassLabel(names=['Negative', 'Neutral', 'Pos

In [ ]:
#make func to turn text into tokens and then use the hugging face dataset .map with that func to get tokenized dataset
def tokenize_the_text(persent_text):
    tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
    tokenized = tokenizer(
        persent_text["MASKED_DOCUMENT"],
        padding="max_length",
        truncation=True,
        max_length=512
    )
    # add the labels
    tokenized["labels"] = persent_text["TRUE_SENTIMENT"]
    return tokenized
#didnt make a datasetloader class like in hw3 since this does the same thing
tokenized_ds = ds.map(tokenize_the_text, batched=True)


Map:   0%|          | 0/3355 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/579 [00:00<?, ? examples/s]

Map:   0%|          | 0/827 [00:00<?, ? examples/s]

Map:   0%|          | 0/578 [00:00<?, ? examples/s]

In [ ]:
print(tokenized_ds["train"][0])
print(tokenized_ds["train"][1])
print(tokenized_ds["train"][2])

{'DOCUMENT_INDEX': 1, 'TITLE': 'German bank LBBW wins EU bailout approval', 'TARGET_ENTITY': 'Landesbank Baden Wuertemberg', 'DOCUMENT': "Germany's Landesbank Baden Wuertemberg won EU approval Tuesday for a state bailout after it promised to shrink its balance sheet by 40 percent and refocus on lending to companies.\n The bank was several state-owned German institutions to run into trouble last year after it ran up more huge losses from investing in high-risk proprietary trading and capital market activities -- a business the EU has now told it to shun.\n Seven current and former managers of the bank are also being investigated by German authorities for risking or damaging the bank's capital by carrying out or failing to block investments in high-risk deals worth hundreds of millions from 2006.\n The European Commission said its Tuesday approval for the state rescue of the bank and its new restructuring plan would allow it become a viable business again -- and that the cutbacks would h

In [ ]:
train_data = tokenized_ds["train"]
val_data = tokenized_ds["validation"]
test_random_data = tokenized_ds["test_random"]
test_fixed_data = tokenized_ds["test_fixed"]

In [ ]:
#make trainer, we're using huggingface Trainer

learning_rate = 3e-5
batch_size = 16
num_epochs = 3 #we start with these but we will change during hyperparam tuning

#we evaluate training at the end of every epoch
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=learning_rate,
    num_train_epochs=num_epochs,
    weight_decay=.01,
    logging_dir="./logs",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to='none' #turn off the weird cloud api thing
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
bert_model = BERT()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#we need to make the evaluation metrics function ourselves for the hugging face trainer
def eval_metrics(preds_and_labels):
    preds, labels = preds_and_labels #grab the predictions and labels from the tuple argument
    preds = preds.argmax(-1) #grab largest probability
    accuracy = accuracy_score(labels, preds) #avg =weighted for multiclass, ecept for accuracy
    precision = precision_score(labels, preds, average='weighted')
    recall = recall_score(labels, preds, average='weighted')
    f1 = f1_score(labels, preds, average = "weighted")
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [ ]:
trainer = Trainer(
    model= bert_model.get_model(),
    args = training_args,
    train_dataset = train_data,
    eval_dataset = val_data,
    tokenizer = bert_model.get_tokenizer(),
    compute_metrics = eval_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.857607,0.551903,0.468105,0.551903,0.442706
2,No log,0.866500,0.600346,0.535726,0.600346,0.565529


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.857607,0.551903,0.468105,0.551903,0.442706
2,No log,0.866500,0.600346,0.535726,0.600346,0.565529
3,0.841600,0.887940,0.586505,0.567247,0.586505,0.571116


TrainOutput(global_step=630, training_loss=0.8017028203086247, metrics={'train_runtime': 228.9257, 'train_samples_per_second': 43.966, 'train_steps_per_second': 2.752, 'total_flos': 2648260326666240.0, 'train_loss': 0.8017028203086247, 'epoch': 3.0})

In [ ]:
#these values, validation loss increased over time.
#acc, recall, and precision increase minutely across epochs 1 to 2, dip back a little in 3 for accuracy and recall
#also i noticed that it set zero division = to 0.0 by default in training to handle the edge case, so i feel no need to set it
#explicitly since i wouldve just set it to 0 as well. I dont want to rerun again

In [ ]:
trainer.evaluate()

{'eval_loss': 0.8879402875900269,
 'eval_accuracy': 0.5865051903114187,
 'eval_precision': 0.5672472797685724,
 'eval_recall': 0.5865051903114187,
 'eval_f1': 0.5711156611400704,
 'eval_runtime': 4.1854,
 'eval_samples_per_second': 138.098,
 'eval_steps_per_second': 8.84,
 'epoch': 3.0}

# Hyperparam Tuning

In [ ]:
#we tune lr first

In [ ]:
learning_rate = 2e-5 # we try lower lr this time
batch_size = 16
num_epochs = 3

bert_model = BERT()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results/lr-tuning-1",
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=learning_rate,
    num_train_epochs=num_epochs,
    weight_decay=.01,
    logging_dir="./logs",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to='none' #turn off the weird cloud api thing
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model= bert_model.get_model(),
    args = training_args,
    train_dataset = train_data,
    eval_dataset = val_data,
    tokenizer = bert_model.get_tokenizer(),
    compute_metrics = eval_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.851556,0.555363,0.477638,0.555363,0.484703
2,No log,0.900120,0.588235,0.520926,0.588235,0.527807
3,0.855400,0.876995,0.589965,0.572458,0.589965,0.569098


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TrainOutput(global_step=630, training_loss=0.8239102439274864, metrics={'train_runtime': 225.1226, 'train_samples_per_second': 44.709, 'train_steps_per_second': 2.798, 'total_flos': 2648260326666240.0, 'train_loss': 0.8239102439274864, 'epoch': 3.0})

In [ ]:
trainer.evaluate()

{'eval_loss': 0.8769952654838562,
 'eval_accuracy': 0.5899653979238755,
 'eval_precision': 0.5724579762114218,
 'eval_recall': 0.5899653979238755,
 'eval_f1': 0.5690978430893591,
 'eval_runtime': 4.1934,
 'eval_samples_per_second': 137.837,
 'eval_steps_per_second': 8.823,
 'epoch': 3.0}

In [ ]:
#lowering lr had slightly better validation loss and slightly worse f1 althought it isnt very significant

In [ ]:
learning_rate = 3e-5
batch_size = 32 #raise batch size
num_epochs = 3

bert_model= BERT()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results/bs-tuning-1",
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=learning_rate,
    num_train_epochs=num_epochs,
    weight_decay=.01,
    logging_dir="./logs",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to='none' #turn off the weird cloud api thing
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model= bert_model.get_model(),
    args = training_args,
    train_dataset = train_data,
    eval_dataset = val_data,
    tokenizer = bert_model.get_tokenizer(),
    compute_metrics = eval_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.841631,0.586505,0.515147,0.586505,0.542587
2,No log,0.835010,0.595156,0.525818,0.595156,0.554655
3,No log,0.848632,0.595156,0.596288,0.595156,0.564100


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TrainOutput(global_step=315, training_loss=0.8361533513144841, metrics={'train_runtime': 213.6785, 'train_samples_per_second': 47.103, 'train_steps_per_second': 1.474, 'total_flos': 2648260326666240.0, 'train_loss': 0.8361533513144841, 'epoch': 3.0})

In [ ]:
#after raising batch size, our training loss stayed roughly the same although we had a smaller validation loss.
#These parameters may perform a litte better by about .03 against unseen data

In [ ]:
trainer.evaluate()

{'eval_loss': 0.8486323952674866,
 'eval_accuracy': 0.5951557093425606,
 'eval_precision': 0.596288027338061,
 'eval_recall': 0.5951557093425606,
 'eval_f1': 0.5641002623166267,
 'eval_runtime': 3.9265,
 'eval_samples_per_second': 147.204,
 'eval_steps_per_second': 4.839,
 'epoch': 3.0}

In [ ]:
learning_rate = 3e-5
batch_size = 16
num_epochs = 5 #more epocs

bert_model = BERT()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results/epoch-tuning-1",
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=learning_rate,
    num_train_epochs=num_epochs,
    weight_decay=.01,
    logging_dir="./logs",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to='none' #turn off the weird cloud api thing
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model= bert_model.get_model(),
    args = training_args,
    train_dataset = train_data,
    eval_dataset = val_data,
    tokenizer = bert_model.get_tokenizer(),
    compute_metrics = eval_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.852885,0.574394,0.502564,0.574394,0.503148
2,No log,0.860565,0.589965,0.519481,0.589965,0.542083
3,0.842200,0.965785,0.577855,0.551656,0.577855,0.555807


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.852885,0.574394,0.502564,0.574394,0.503148
2,No log,0.860565,0.589965,0.519481,0.589965,0.542083
3,0.842200,0.965785,0.577855,0.551656,0.577855,0.555807
4,0.842200,1.166863,0.574394,0.539260,0.574394,0.547595
5,0.409900,1.300599,0.567474,0.539227,0.567474,0.546696


TrainOutput(global_step=1050, training_loss=0.6062064888363793, metrics={'train_runtime': 374.9547, 'train_samples_per_second': 44.739, 'train_steps_per_second': 2.8, 'total_flos': 4413767211110400.0, 'train_loss': 0.6062064888363793, 'epoch': 5.0})

In [ ]:
#validation loss was much higher for no improvement in f1, not great

In [ ]:
learning_rate = 3e-5
batch_size = 8 #cut batch size in half
num_epochs = 3

bert_model = BERT()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results/bs-tuning-2",
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=learning_rate,
    num_train_epochs=num_epochs,
    weight_decay=.01,
    logging_dir="./logs",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to='none' #turn off the weird cloud api thing
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model= bert_model.get_model(),
    args = training_args,
    train_dataset = train_data,
    eval_dataset = val_data,
    tokenizer = bert_model.get_tokenizer(),
    compute_metrics = eval_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.855750,0.558824,0.477817,0.558824,0.491332
2,0.910700,0.924321,0.562284,0.555297,0.562284,0.507815
3,0.738400,1.079653,0.557093,0.536232,0.557093,0.541304


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TrainOutput(global_step=1260, training_loss=0.7693514869326636, metrics={'train_runtime': 241.3092, 'train_samples_per_second': 41.71, 'train_steps_per_second': 5.222, 'total_flos': 2648260326666240.0, 'train_loss': 0.7693514869326636, 'epoch': 3.0})

In [ ]:
#validation loss still higher than original and f1 score is lower. did i hit the sweetspot at the start??

In [ ]:
trainer.evaluate()

{'eval_loss': 1.0796531438827515,
 'eval_accuracy': 0.5570934256055363,
 'eval_precision': 0.5362317616740039,
 'eval_recall': 0.5570934256055363,
 'eval_f1': 0.5413039481884421,
 'eval_runtime': 4.4017,
 'eval_samples_per_second': 131.313,
 'eval_steps_per_second': 16.585,
 'epoch': 3.0}

In [ ]:
learning_rate = 2e-5 #lower lr
batch_size = 32 #raise batch size
num_epochs = 3

bert_model= BERT()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
training_args = TrainingArguments(
    output_dir="./results/lr_bs-tuning",
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=learning_rate,
    num_train_epochs=num_epochs,
    weight_decay=.01,
    logging_dir="./logs",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to='none' #turn off the weird cloud api thing
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model= bert_model.get_model(),
    args = training_args,
    train_dataset = train_data,
    eval_dataset = val_data,
    tokenizer = bert_model.get_tokenizer(),
    compute_metrics = eval_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.853205,0.570934,0.493297,0.570934,0.505551


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.853205,0.570934,0.493297,0.570934,0.505551
2,No log,0.838815,0.591696,0.520550,0.591696,0.545738
3,No log,0.849179,0.591696,0.529090,0.591696,0.558136


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TrainOutput(global_step=315, training_loss=0.8501426091269841, metrics={'train_runtime': 212.3619, 'train_samples_per_second': 47.396, 'train_steps_per_second': 1.483, 'total_flos': 2648260326666240.0, 'train_loss': 0.8501426091269841, 'epoch': 3.0})

In [ ]:
# the models are roughly simmilar thought soley changing bacth size to 32 provided better validation loss
#and cost very minimal f1

In [ ]:
learning_rate = 3e-5
batch_size = 32
num_epochs = 3 #we start with these but we will change during hyperparam tuning

best_model = BERT()

#we evaluate training at the end of every epoch
training_args = TrainingArguments(
    output_dir="./results/best-model-1",
    evaluation_strategy="epoch",
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=learning_rate,
    num_train_epochs=num_epochs,
    weight_decay=.01,
    logging_dir="./logs",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    report_to='none' #turn off the weird cloud api thing
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
#hyperparameter tuning has not changed too much, although i think the best model  is batch = 32, lr = 3e-5, epoch=3

In [ ]:
trainer = Trainer(
    model= best_model.get_model(),
    args = training_args,
    train_dataset = train_data,
    eval_dataset = val_data,
    tokenizer = best_model.get_tokenizer(),
    compute_metrics = eval_metrics
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,No log,0.847768,0.602076,0.530810,0.602076,0.557342
2,No log,0.847159,0.607266,0.536290,0.607266,0.563937
3,No log,0.848468,0.595156,0.557502,0.595156,0.568482


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TrainOutput(global_step=315, training_loss=0.8393699040488591, metrics={'train_runtime': 213.2434, 'train_samples_per_second': 47.2, 'train_steps_per_second': 1.477, 'total_flos': 2648260326666240.0, 'train_loss': 0.8393699040488591, 'epoch': 3.0})

#Testing

In [ ]:
test_result_random = trainer.evaluate(eval_dataset=test_random_data)

In [ ]:
print(test_result_random)

{'eval_loss': 0.8950446248054504, 'eval_accuracy': 0.5734024179620034, 'eval_precision': 0.5615065795225828, 'eval_recall': 0.5734024179620034, 'eval_f1': 0.5498887627230978, 'eval_runtime': 3.9489, 'eval_samples_per_second': 146.622, 'eval_steps_per_second': 4.811, 'epoch': 3.0}


In [ ]:
test_result_fixed = trainer.evaluate(eval_dataset=test_fixed_data)
print(test_result_fixed)

{'eval_loss': 1.0384021997451782, 'eval_accuracy': 0.4909310761789601, 'eval_precision': 0.4760043026639593, 'eval_recall': 0.4909310761789601, 'eval_f1': 0.45912873838434015, 'eval_runtime': 5.5964, 'eval_samples_per_second': 147.774, 'eval_steps_per_second': 4.646, 'epoch': 3.0}


In [ ]:
trainer.save_model('./idea1-best-model')

In [ ]:
!zip -r idea1-best-model.zip ./idea1-best-model

  adding: idea1-best-model/ (stored 0%)
  adding: idea1-best-model/tokenizer_config.json (deflated 76%)
  adding: idea1-best-model/special_tokens_map.json (deflated 42%)
  adding: idea1-best-model/tokenizer.json (deflated 71%)
  adding: idea1-best-model/model.safetensors (deflated 7%)
  adding: idea1-best-model/training_args.bin (deflated 51%)
  adding: idea1-best-model/config.json (deflated 52%)
  adding: idea1-best-model/vocab.txt (deflated 53%)


#Trying different sentences

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!unzip idea1-best-model.zip -d ./best_model

Archive:  idea1-best-model.zip
   creating: ./best_model/idea1-best-model/
  inflating: ./best_model/idea1-best-model/tokenizer_config.json  
  inflating: ./best_model/idea1-best-model/special_tokens_map.json  
  inflating: ./best_model/idea1-best-model/tokenizer.json  
  inflating: ./best_model/idea1-best-model/model.safetensors  
  inflating: ./best_model/idea1-best-model/training_args.bin  
  inflating: ./best_model/idea1-best-model/config.json  
  inflating: ./best_model/idea1-best-model/vocab.txt  


In [ ]:
model_path = "./best_model/idea1-best-model"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

NameError: name 'AutoTokenizer' is not defined

In [ ]:
sentence="The phone is awesome and works as very well!"
tokenized_sentence = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

In [ ]:
with torch.no_grad():
    outputs = model(**tokenized_sentence)

logits = outputs.logits
predicted_class = torch.argmax(logits, dim=1).item()
print(f"Predicted Class -1: no label , 0, bad, 1, is neutral, 2 is good: {predicted_class}")

Predicted Class -1: no label , 0, bad, 1, is neutral, 2 is good: 2


In [ ]:
sentences = [
    "The phone is awesome and works as very well!",
    "This place sucks and I hope it fails.",
    "The item is okay, not great but not bad either.",
    "The delivery guy was fast, however the food is cold and the box was dented.",
    "I can't even begin to describe what I think about this place.",
    "The hotel was beautiful but the manager was rude and rolled her eyes at me.",
    "The hotel is evil, disgusting, horrible, yucky, rude, "
]

tokenized_sentences = tokenizer(sentences, return_tensors="pt", truncation=True, padding=True, max_length=512).to(device)
outputs = model(**tokenized_sentences)
predictions = torch.argmax(outputs.logits, dim=1)

for sentence, prediction in zip(sentences, predictions):
    print(f"Sentence: {sentence}")
    print(f"Predicted Class (-1: no label, 0: bad, 1: neutral, 2: good): {prediction.item()}\n")


Sentence: The phone is awesome and works as very well!
Predicted Class (-1: no label, 0: bad, 1: neutral, 2: good): 2

Sentence: This place sucks and I hope it fails.
Predicted Class (-1: no label, 0: bad, 1: neutral, 2: good): 1

Sentence: The item is okay, not great but not bad either.
Predicted Class (-1: no label, 0: bad, 1: neutral, 2: good): 1

Sentence: The delivery guy was fast, however the food is cold and the box was dented.
Predicted Class (-1: no label, 0: bad, 1: neutral, 2: good): 1

Sentence: I can't even begin to describe what I think about this place.
Predicted Class (-1: no label, 0: bad, 1: neutral, 2: good): 2

Sentence: The hotel was beautiful but the manager was rude and rolled her eyes at me.
Predicted Class (-1: no label, 0: bad, 1: neutral, 2: good): 1

Sentence: The hotel is evil, disgusting, horrible, yucky, rude, 
Predicted Class (-1: no label, 0: bad, 1: neutral, 2: good): 0

